In [2]:
import sys
sys.path.insert(0, os.path.abspath('../'))
import os
import pyspark as spark
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
import pyspark.sql.functions as f
from pyspark.sql import DataFrame, Row
from functools import reduce
import functools
from pyspark.sql.types import StructType, StructField, StringType, LongType, ArrayType, IntegerType, MapType
from pyspark.sql.functions import *

In [3]:
from pyspark import SparkContext
from pyspark import SparkConf
from pyspark.sql import SQLContext
conf=SparkConf()
conf.set("spark.executor.memory", "4g")
conf.set("spark.driver.memory", "4g")
conf.set("spark.cores.max", "6")

sc = SparkContext.getOrCreate(conf)

spark = SQLContext(sc)

22/12/16 13:04:09 WARN Utils: Your hostname, preacher resolves to a loopback address: 127.0.1.1; using 192.168.0.102 instead (on interface enp4s0)
22/12/16 13:04:09 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/12/16 13:04:10 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
/home/preacher/anaconda3/envs/deltarun/lib/python3.9/site-packages/pyspark/sql/context.py:77: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [35]:
bungeedatalake_path = "/home/preacher/Bungee/bnt/data_set/bungeedatalake/office_supplies"
exact_matches_path = "/home/preacher/Bungee/bnt/data_set/match_data_warehouse/first_run"

In [ ]:
bungeedatalake = spark.read.format('csv').option('header','true').load(bungeedatalake_path)
print(bungeedatalake.count())
bungeedatalake.show()


In [ ]:
exact_matches = spark.read.format('csv').option('header','true').load(exact_matches_path)
print(exact_matches.count())
exact_matches.show()

In [110]:
def save_df(df, prefix):
    path = '/home/preacher/Bungee/bnt/data_set/'+prefix
    print("Saving df to loc = {}".format(path))
    df.write.mode("overwrite").option("header",True).parquet(path)
    print("dfs saved successfully")

In [40]:
def get_brand_matches(exact_matches, bungeedatalake):
    uuid_a_to_brand = exact_matches.join(bungeedatalake, bungeedatalake.uuid == exact_matches.uuid_a, 'inner').select('uuid_a', 'uuid_b', col("brand").alias("brand_a"), col("source_store").alias("source_store_a"))
    uuid_a_b_to_brand = uuid_a_to_brand.join(bungeedatalake, bungeedatalake.uuid == uuid_a_to_brand.uuid_b, 'inner').select('uuid_a', 'uuid_b', 'brand_a', 'source_store_a', col("brand").alias("brand_b"), col("source_store").alias("source_store_b"))
    brand_matches = uuid_a_b_to_brand.select('uuid_a', 'uuid_b','brand_a', 'brand_b', 'source_store_a', 'source_store_b').na.drop()
    brand_matches.printSchema()
    return brand_matches

In [41]:
branbd_matches = get_brand_matches(exact_matches, bungeedatalake)

root
 |-- uuid_a: string (nullable = true)
 |-- uuid_b: string (nullable = true)
 |-- brand_a: string (nullable = true)
 |-- brand_b: string (nullable = true)
 |-- source_store_a: string (nullable = true)
 |-- source_store_b: string (nullable = true)



In [42]:
branbd_matches.show()

+--------------------+--------------------+--------------------+--------------------+---------------+---------------+
|              uuid_a|              uuid_b|             brand_a|             brand_b| source_store_a| source_store_b|
+--------------------+--------------------+--------------------+--------------------+---------------+---------------+
|001034383213<>386...|00010343832138<>2...|               Epson|INGRAM MICRO D, INC.|walmart_walmart|staples_staples|
|11974320827<>1670...|00011974320827<>b...|GARVEY PRODUCTS, ...|              GARVEY|staples_staples|  amazon_amazon|
|11974320827<>1670...|00011974320827<>b...|GARVEY PRODUCTS, ...|              GARVEY|staples_staples|  amazon_amazon|
|013803078077<>563...|00013803078077<>3...|              Canon®|DIVERSIFIED COMPU...|wbmason_wbmason|staples_staples|
|013803078077<>149...|00013803078077<>3...|               Canon|DIVERSIFIED COMPU...|walmart_walmart|staples_staples|
|sku_b000w7r13a<>b...|00013803078077<>3...|             

In [23]:
save_df(branbd_matches, 'brand_matches')

NameError: name 'save_df' is not defined

In [74]:
def combine_dfs(df_list):
    print("Combning dfs")
    combined_df = reduce(lambda df1, df2: df1.union(df2.select(df1.columns)), df_list)
    # print("Combined df size = {}".format(combined_df.count()))
    return combined_df

In [46]:
def get_max_id(df):
    result = df.select([max('id')])
    max_id = result.collect()[0]['max(id)']
    return max_id

def insert_id( df, start_idx):
    print("Getting ids for new uuids")
    index = start_idx
    updated_schema = StructType(df.schema.fields[:] + [StructField("id", LongType(), False)])
    zipped_rdd = df.rdd.zipWithIndex().map(lambda x: (x[0], x[1] + index))
    id_df = (zipped_rdd.map(lambda ri: Row(*list(ri[0]) + [ri[1]])).toDF(updated_schema))
    # print("New id_uuid = {}".format(new_id_uuid_df.count()))
    return id_df

In [31]:
def print_df(df):
    df.printSchema()
    print(df.count())

VERTEX GENERATION

In [45]:

def generate_brand_to_store_set(brand_matches):
    # convert brand to id 
    brand_a = brand_matches.select(col("brand_a").alias("brand"), col("source_store_a").alias("source_store"))
    # print(brand_a.count())
    brand_b = brand_matches.select(col("brand_b").alias("brand"), col("source_store_b").alias("source_store"))
    # print(brand_b.count())
    brand_to_store = combine_dfs([brand_a, brand_b])
    # print(brand_to_store.count())
    brand_to_store.printSchema()
    brand_to_store_set = brand_to_store.groupBy('brand').agg(collect_set('source_store').alias("source_store_list"))
    brand_to_store_set.printSchema()
    return brand_to_store_set

In [46]:
def get_vertex(branbd_matches):
        print("Fetching vertex")
        # print(branbd_matches.count())
        brand_to_store = generate_brand_to_store_set(branbd_matches)
        # if self.run_type == 'DAILY':
        #     old_vertices = get_data_from_s3(self.s3_bucket, self.vertex_path)
        #     new_brands = brand_to_store.join(old_vertices, old_vertices.brand == brand_to_store.brand, "leftanti")
        #     last_id = get_max_id(old_vertices)
        #     new_vertices = insert_id(new_brands, last_id)
        #     self.vertices = combine_dfs([old_vertices, new_vertices])
        #     # delete old data
        # else:
        # print(brand_to_store.count())
        vertices = insert_id(brand_to_store, 0)
        # print(vertices.show())
        # vertices.show()
        print("Fetching vertex finishes")
        return vertices
        

In [ ]:
vertex = get_vertex(branbd_matches)


EDGE GENERATION

In [74]:
def generate_graph_edges(brand_matches, vertices):
    brand_matches = brand_matches.select('brand_a', 'brand_b')
    brand_matches = brand_matches.groupBy('brand_a', 'brand_b').count()
    # calculating weights
    directed_edge_a_b = brand_matches.select('brand_a','brand_b', 'count')
    directed_edge_b_a = brand_matches.select(col('brand_a').alias('brand_b') ,col('brand_b').alias('brand_a'), 'count')
    undirected_edges = combine_dfs([directed_edge_a_b, directed_edge_b_a])
    undirected_edges.show()
    vertex_a_id = undirected_edges.join(vertices, vertices.brand == undirected_edges.brand_a, 'inner').select(col('id').alias('src'), 'brand_b','count')
    undirected_id_edges = vertex_a_id.join(vertices, vertices.brand == vertex_a_id.brand_b, 'inner').select(col('id').alias('dst'), 'src', 'count')
    return undirected_id_edges

In [75]:
def get_edge(brand_matches, vertex):
        new_edges = generate_graph_edges(brand_matches, vertex)
        # if self.run_type == 'DAILY':
        #     old_edges = get_data_from_s3(self.s3_bucket, self.edge_path)
        #     edges = combine_dfs([old_edges, new_edges])
        #     # delete old data
        #     saving_df_to_s3(edges, self.s3_bucket, self.vertex_path)
        #     self.edges = edges
        # else:
        edges =  new_edges
        return new_edges

        # saving_df_to_s3(self.edges, self.s3_bucket, self.vertex_path)

In [76]:
branbd_matches

DataFrame[uuid_a: string, uuid_b: string, brand_a: string, brand_b: string, source_store_a: string, source_store_b: string]

In [77]:
edges = get_edge(branbd_matches, vertex)

Combning dfs


+--------------------+--------------------+-----+
|             brand_a|             brand_b|count|
+--------------------+--------------------+-----+
|         Sargent Art|         Sargent Art|  113|
|             Moldex®|              Moldex|   16|
|              Avery®|               Avery| 1904|
|            Deflecto|            Deflecto|  173|
|           Essendant|              Canon®|    3|
|           Colombina|           Colombina|    7|
|               Ghent|GHENT MANUFACTURI...|  140|
|            Fadeless|DIXON TICONDEROGA...|   23|
|                 GBC|           Swingline|   35|
|              PayDay|          Hershey's®|    1|
|          ACCUSTAMP®|           AccuStamp|    1|
|                 BBF|BUSH INDUSTRIES, INC|  209|
|                VARI|BUSH INDUSTRIES, INC|    3|
|              Marcal|              Marcal|    3|
|PERFORMANCE FOOD GRP|          Coffeemate|   15|
|    PETRA INDUSTRIES|         Range Kleen|    7|
|Coastwide Profess...|          MIDLAB INC|   18|


In [78]:
edges.show()
edges.count()

+-----+----+-----+
|  dst| src|count|
+-----+----+-----+
| 4497|4497|  113|
| 4015|4016|   16|
| 7932| 243| 1904|
| 8450|8450|  173|
| 8201|8637|    3|
|  532| 532|    7|
| 3432|3467|  140|
| 3136|3371|   23|
| 7259|3425|   35|
| 1049|9755|    1|
|  113|2577|    1|
| 2778|7947|  209|
| 2778|4856|    3|
| 9377|9377|    3|
| 8297|9707|   15|
| 4344|1627|    7|
| 3912|8287|   18|
| 2775|8125|    9|
|10022|6812|    8|
| 8708|8711|  147|
+-----+----+-----+
only showing top 20 rows



82250

In [81]:
#  vertex_neighbour  calculation
vertex_neighbour = edges.groupBy('src').agg(collect_set('dst').alias("neighbour"))

In [84]:

vertex_neighbour.show(1000)
vertex_neighbour.count()

+----+--------------------+
| src|           neighbour|
+----+--------------------+
|   1|[2599, 7384, 1032...|
|   2|       [10774, 7940]|
|   3|        [6012, 2753]|
|   4|              [3631]|
|   5|  [9271, 1305, 6361]|
|   6|[5077, 4474, 1015...|
|   7|              [7480]|
|   8|              [8287]|
|   9|              [8287]|
|  10|              [8287]|
|  11|        [1657, 6776]|
|  12|              [2526]|
|  13|[7315, 4122, 2046...|
|  14|[4190, 9905, 7209...|
|  15|              [6939]|
|  16|              [2753]|
|  17|              [8287]|
|  18|              [7209]|
|  19|              [7480]|
|  20|              [7480]|
|  21| [3581, 10641, 3285]|
|  22|              [1292]|
|  23|              [2909]|
|  24|[2941, 5483, 8172...|
|  25|  [1908, 2046, 7209]|
|  26|             [10774]|
|  27|              [7209]|
|  28|             [10774]|
|  29|              [8504]|
|  30|             [10641]|
|  31|              [3249]|
|  32|[763, 1657, 764, ...|
|  33|              

10829

In [59]:
weighted_edges.count()

41125

In [ ]:
print_df(edges)
vertex_a_id = edges.join(vertex, vertex.brand == edges.brand_a, 'inner').select(col('id').alias('src'), 'brand_b')
print_df(vertex_a_id)
edge = vertex_a_id.join(vertex, vertex.brand == vertex_a_id.brand_b, 'inner').select(col('id').alias('dst'), 'src')
print_df(edge)

In [ ]:
edge.show()

In [ ]:
vertex.show(100)

GRAPH PREPARATION

In [36]:
from graphframes import *

In [37]:
graph =  GraphFrame(vertex, edge)

In [38]:
type(graph)

graphframes.graphframe.GraphFrame

GENERATE_BRAND_SETS

In [39]:
sc.setCheckpointDir("/home/preacher/Bungee/bnt/checkpoints/")
cc = graph.connectedComponents()

In [ ]:
cc.show(1000)

In [42]:
cc.count()

10829

In [33]:
cc = cc.withColumnRenamed('collect_set(source_store)', 'source_store')

In [34]:
cc.write.parquet('/home/preacher/Bungee/bnt/cc.parquet')

In [43]:
brand_id_sets = cc.groupBy('component').agg(collect_set('id').alias("brand_id_list"))

In [44]:
brand_sets = cc.groupBy('component').agg(collect_set('brand').alias("brand_list"))

In [ ]:
brand_id_sets.show()

In [ ]:
brand_sets.show(1000)

In [48]:
brand_sets.count()

1001

In [54]:
brand_id_sets = brand_id_sets.withColumn("brand_id_list", col("brand_id_list").cast("string"))

In [52]:
save_df(brand_sets, 'brand_sets')

Saving df to loc = /home/preacher/Bungee/bnt/data_set/brand_sets
dfs saved successfully


In [55]:
save_df(brand_id_sets, 'brand_id_sets')

Saving df to loc = /home/preacher/Bungee/bnt/data_set/brand_id_sets
dfs saved successfully


BRAND_SET_COMPARISON

In [3]:
def open_file(path):
    df = spark.read.format('csv').option('header','true').load(path)
    print(df.count())
    df.show()
    return df

In [4]:
old_cc_path = '/home/preacher/Bungee/bnt/data_set/old_brand_id_set.csv'
new_cc_path = '/home/preacher/Bungee/bnt/data_set/new_brand_id_set.csv'

In [5]:
old_brand_set = open_file(old_cc_path)
new_brand_set = open_file(new_cc_path)

8
+----+------------+
|  id|brand_id_set|
+----+------------+
|1000|       6,2,1|
|1001|      7,8,10|
|1002|    12,15,17|
|1003|        9,11|
|1004| 100,101,102|
|1005|         4,5|
|1006|    21,22,23|
|1007|    31,32,33|
+----+------------+

7
+------------+
|brand_id_set|
+------------+
| 100,101,102|
|    97,98,99|
|    7,8,9,10|
| 12,15,16,17|
|         4,6|
|       22,23|
|    32,33,34|
+------------+



In [10]:
old_brand_set.printSchema()

root
 |-- brand_id_set: string (nullable = true)



In [16]:
# convert string to array
def convert_string_to_sorted_array(df, col_name):
    df = df.select(split(col(col_name),",").alias('array')).drop(col_name)
    df = df.select("array",sort_array("array",asc=True).alias(col_name)).drop('array')
    df.show()
    return df

In [17]:
# convert array and string
def convert_array_to_string(df):
    df = df.withColumn("brand_id_string",concat_ws(",",col("brand_id_set"))).drop('brand_id_set')
    df.show()
    return df

In [14]:
old_brand_id_set = convert_string_to_sorted_array(old_brand_set, 'brand_id_set')
new_brand_id_set = convert_string_to_sorted_array(new_brand_set, 'brand_id_set')

+---------------+
|   brand_id_set|
+---------------+
|      [1, 2, 6]|
|     [10, 7, 8]|
|   [12, 15, 17]|
|            [9]|
|[100, 101, 102]|
|         [4, 5]|
|   [21, 22, 23]|
|   [31, 32, 33]|
+---------------+

+----------------+
|    brand_id_set|
+----------------+
| [100, 101, 102]|
|    [97, 98, 99]|
|   [10, 7, 8, 9]|
|[12, 15, 16, 17]|
|          [4, 6]|
|        [22, 23]|
|    [32, 33, 34]|
+----------------+



In [15]:
old_brand_id_string = convert_array_to_string(old_brand_id_set)
new_brand_id_string = convert_array_to_string(new_brand_id_set)

+---------------+
|brand_id_string|
+---------------+
|          1,2,6|
|         10,7,8|
|       12,15,17|
|              9|
|    100,101,102|
|            4,5|
|       21,22,23|
|       31,32,33|
+---------------+

+---------------+
|brand_id_string|
+---------------+
|    100,101,102|
|       97,98,99|
|       10,7,8,9|
|    12,15,16,17|
|            4,6|
|          22,23|
|       32,33,34|
+---------------+



In [16]:
updated_and_new_set  = new_brand_id_string.subtract(old_brand_id_string)

In [17]:
updated_and_new_set.show()

+---------------+
|brand_id_string|
+---------------+
|    12,15,16,17|
|       10,7,8,9|
|       97,98,99|
|          22,23|
|            4,6|
|       32,33,34|
+---------------+



In [18]:
updated_and_new_set = convert_string_to_sorted_array(updated_and_new_set,'brand_id_string')

+----------------+
| brand_id_string|
+----------------+
|[12, 15, 16, 17]|
|   [10, 7, 8, 9]|
|    [97, 98, 99]|
|        [22, 23]|
|          [4, 6]|
|    [32, 33, 34]|
+----------------+



In [19]:
old_set = convert_string_to_sorted_array(old_brand_id_string,'brand_id_string')

+---------------+
|brand_id_string|
+---------------+
|      [1, 2, 6]|
|     [10, 7, 8]|
|   [12, 15, 17]|
|            [9]|
|[100, 101, 102]|
|         [4, 5]|
|   [21, 22, 23]|
|   [31, 32, 33]|
+---------------+



In [19]:
old_set.printSchema()

root
 |-- brand_id_string: array (nullable = false)
 |    |-- element: string (containsNull = true)



In [24]:
old_set = insert_id(old_set, 0)
new_set = insert_id(updated_and_new_set, 0)

Getting ids for new uuids
Getting ids for new uuids


In [25]:
100000000 
old_set.show()
4000 
new_set.show()

+---------------+---+
|brand_id_string| id|
+---------------+---+
|      [1, 2, 6]|  1|
|     [10, 7, 8]|  2|
|   [12, 15, 17]|  3|
|            [9]|  4|
|[100, 101, 102]|  5|
|         [4, 5]|  6|
|   [21, 22, 23]|  7|
|   [31, 32, 33]|  8|
+---------------+---+

+----------------+---+
| brand_id_string| id|
+----------------+---+
|[12, 15, 16, 17]|  1|
|   [10, 7, 8, 9]|  2|
|    [97, 98, 99]|  3|
|        [22, 23]|  4|
|          [4, 6]|  5|
|    [32, 33, 34]|  6|
+----------------+---+



In [ ]:
100040000

In [26]:
id_set_num = old_set.select(old_set.id,explode(old_set.brand_id_string).alias('brand'))
id_set_num = id_set_num.withColumnRenamed('id','set_id_old')
id_set_num.show()

+----------+-----+
|set_id_old|brand|
+----------+-----+
|         1|    1|
|         1|    2|
|         1|    6|
|         2|   10|
|         2|    7|
|         2|    8|
|         3|   12|
|         3|   15|
|         3|   17|
|         4|    9|
|         5|  100|
|         5|  101|
|         5|  102|
|         6|    4|
|         6|    5|
|         7|   21|
|         7|   22|
|         7|   23|
|         8|   31|
|         8|   32|
+----------+-----+
only showing top 20 rows



In [27]:
new_id_set_num = new_set.select(new_set.id,explode(new_set.brand_id_string).alias('brand'))
new_id_set_num = new_id_set_num.withColumnRenamed('id','set_id_new')
new_id_set_num.show()

+----------+-----+
|set_id_new|brand|
+----------+-----+
|         1|   12|
|         1|   15|
|         1|   16|
|         1|   17|
|         2|   10|
|         2|    7|
|         2|    8|
|         2|    9|
|         3|   97|
|         3|   98|
|         3|   99|
|         4|   22|
|         4|   23|
|         5|    4|
|         5|    6|
|         6|   32|
|         6|   33|
|         6|   34|
+----------+-----+



In [28]:
old_id_set_num = id_set_num

In [30]:
#updated brands
new_id_set_num = new_id_set_num.withColumnRenamed('brand','brand_new')
set_compare = new_id_set_num.join(old_id_set_num, new_id_set_num.brand_new == old_id_set_num.brand, 'inner').drop('brand_new')
set_compare.show()

+----------+----------+-----+
|set_id_new|set_id_old|brand|
+----------+----------+-----+
|         2|         2|   10|
|         1|         3|   12|
|         1|         3|   15|
|         1|         3|   17|
|         4|         7|   22|
|         4|         7|   23|
|         6|         8|   32|
|         6|         8|   33|
|         5|         6|    4|
|         5|         1|    6|
|         2|         2|    7|
|         2|         2|    8|
|         2|         4|    9|
+----------+----------+-----+



In [39]:
#new brands
new_id_set_num = new_id_set_num.withColumnRenamed('brand_new','brand')

updated_brands = set_compare.select( 'set_id_new','brand')
updated_brands.show()
new_brands = new_id_set_num.subtract(updated_brands)
new_brands.show()



+----------+-----+
|set_id_new|brand|
+----------+-----+
|         2|   10|
|         1|   12|
|         1|   15|
|         1|   17|
|         4|   22|
|         4|   23|
|         6|   32|
|         6|   33|
|         5|    4|
|         5|    6|
|         2|    7|
|         2|    8|
|         2|    9|
+----------+-----+

+----------+-----+
|set_id_new|brand|
+----------+-----+
|         3|   99|
|         6|   34|
|         3|   98|
|         1|   16|
|         3|   97|
+----------+-----+



In [50]:
brand_old_id = set_compare.withColumn("brand_old_id", struct(col("brand"),col("set_id_old"))).drop('brand','set_id_old')
brand_old_id = brand_old_id.withColumnRenamed('set_id_new','id')
brand_old_id = brand_old_id.groupBy('id').agg(collect_list('brand_old_id').alias("brand_old_id"))
brand_old_id = brand_old_id.withColumnRenamed('id','new_id') 
brand_old_id.show(brand_old_id.count(), False)


+------+---------------------------------+
|new_id|brand_old_id                     |
+------+---------------------------------+
|6     |[{32, 8}, {33, 8}]               |
|5     |[{4, 6}, {6, 1}]                 |
|1     |[{12, 3}, {15, 3}, {17, 3}]      |
|2     |[{10, 2}, {7, 2}, {8, 2}, {9, 4}]|
|4     |[{22, 7}, {23, 7}]               |
+------+---------------------------------+



In [51]:
brand_new_id = set_compare.withColumn("brand_new_id", struct(col("brand"),col("set_id_new"))).drop('brand','set_id_new')
brand_new_id = brand_new_id.withColumnRenamed('set_id_old','id')
brand_new_id = brand_new_id.groupBy('id').agg(collect_list('brand_new_id').alias("brand_new_id"))
brand_new_id = brand_new_id.withColumnRenamed('id','old_id') 
brand_new_id.show(brand_new_id.count(), False)

+------+---------------------------+
|old_id|brand_new_id               |
+------+---------------------------+
|7     |[{22, 4}, {23, 4}]         |
|6     |[{4, 5}]                   |
|1     |[{6, 5}]                   |
|3     |[{12, 1}, {15, 1}, {17, 1}]|
|8     |[{32, 6}, {33, 6}]         |
|2     |[{10, 2}, {7, 2}, {8, 2}]  |
|4     |[{9, 2}]                   |
+------+---------------------------+



In [55]:
brand_old_id = set_compare.groupBy('set_id_old').agg(collect_set('set_id_new').alias("brand_old_id"))
brand_old_id.show(brand_old_id.count(), False)

+----------+------------+
|set_id_old|brand_old_id|
+----------+------------+
|7         |[4]         |
|6         |[5]         |
|1         |[5]         |
|3         |[1]         |
|8         |[6]         |
|2         |[2]         |
|4         |[2]         |
+----------+------------+



In [56]:
brand_old_id = set_compare.groupBy('set_id_new').agg(collect_set('set_id_old').alias("old_id_set"))
brand_old_id.show(brand_old_id.count(), False)

+----------+----------+
|set_id_new|old_id_set|
+----------+----------+
|6         |[8]       |
|5         |[1, 6]    |
|1         |[3]       |
|2         |[2, 4]    |
|4         |[7]       |
+----------+----------+



In [62]:
new_id_brands = brand_old_id.join(new_set, brand_old_id.set_id_new == new_set.id, 'inner').drop('id').withColumnRenamed('brand_id_string','new_brand_sets')
new_id_brands.show()

+----------+----------+----------------+
|set_id_new|old_id_set|  new_brand_sets|
+----------+----------+----------------+
|         6|       [8]|    [32, 33, 34]|
|         5|    [1, 6]|          [4, 6]|
|         1|       [3]|[12, 15, 16, 17]|
|         2|    [2, 4]|   [10, 7, 8, 9]|
|         4|       [7]|        [22, 23]|
+----------+----------+----------------+



In [71]:
new_id_brands_exploded_old_id = new_id_brands.withColumn('set_id_old', explode('old_id_set')).drop('old_id_set','new_brand_sets')
new_id_brands_exploded_old_id.show()

+----------+----------+
|set_id_new|set_id_old|
+----------+----------+
|         6|         8|
|         5|         1|
|         5|         6|
|         1|         3|
|         2|         2|
|         2|         4|
|         4|         7|
+----------+----------+



In [76]:
id_set_num = id_set_num.withColumnRenamed('set_id_old','id')

In [77]:
old_id_brand_old_id_brand = new_id_brands_exploded_old_id.join(id_set_num, new_id_brands_exploded_old_id.set_id_old == id_set_num.id, 'inner').drop('id').withColumnRenamed('brand_id_string','old_brand_sets')
old_id_brand_old_id_brand.show()


+----------+----------+-----+
|set_id_new|set_id_old|brand|
+----------+----------+-----+
|         4|         7|   21|
|         4|         7|   22|
|         4|         7|   23|
|         5|         6|    4|
|         5|         6|    5|
|         5|         1|    1|
|         5|         1|    2|
|         5|         1|    6|
|         1|         3|   12|
|         1|         3|   15|
|         1|         3|   17|
|         6|         8|   31|
|         6|         8|   32|
|         6|         8|   33|
|         2|         2|   10|
|         2|         2|    7|
|         2|         2|    8|
|         2|         4|    9|
+----------+----------+-----+



In [82]:
updates_old_set = old_id_brand_old_id_brand.groupBy('set_id_new').agg(collect_set('brand').alias("old_id_brands"))
updates_old_set.show()

+----------+---------------+
|set_id_new|  old_id_brands|
+----------+---------------+
|         6|   [31, 32, 33]|
|         5|[1, 2, 5, 4, 6]|
|         1|   [17, 12, 15]|
|         2|  [8, 9, 10, 7]|
|         4|   [23, 21, 22]|
+----------+---------------+



In [80]:
new_set.show()

+----------------+---+
| brand_id_string| id|
+----------------+---+
|[12, 15, 16, 17]|  1|
|   [10, 7, 8, 9]|  2|
|    [97, 98, 99]|  3|
|        [22, 23]|  4|
|          [4, 6]|  5|
|    [32, 33, 34]|  6|
+----------------+---+



In [86]:
updated_brand_dets = updates_old_set.join(new_set, updates_old_set.set_id_new == new_set.id, 'inner').drop('set_id_new')
updated_brand_dets.show()

+---------------+----------------+---+
|  old_id_brands| brand_id_string| id|
+---------------+----------------+---+
|   [31, 32, 33]|    [32, 33, 34]|  6|
|[1, 2, 5, 4, 6]|          [4, 6]|  5|
|   [17, 12, 15]|[12, 15, 16, 17]|  1|
|  [8, 9, 10, 7]|   [10, 7, 8, 9]|  2|
|   [23, 21, 22]|        [22, 23]|  4|
+---------------+----------------+---+



In [98]:
res = updated_brand_dets.withColumn("brand_sets", array_union(col("brand_id_string"), col("old_id_brands"))).drop('old_id_brands','brand_id_string')
updated_sets = res
updated_sets.show()

+---+----------------+
| id|      brand_sets|
+---+----------------+
|  6|[32, 33, 34, 31]|
|  5| [4, 6, 1, 2, 5]|
|  1|[12, 15, 16, 17]|
|  2|   [10, 7, 8, 9]|
|  4|    [22, 23, 21]|
+---+----------------+



In [89]:
updated_id_set = res.select('id')
updated_id_set.show()

+---+
| id|
+---+
|  6|
|  5|
|  1|
|  2|
|  4|
+---+



In [ ]:
updated_id_set = res.select('id')
updated_id_set.show()

In [92]:
all_ids = new_set.select('id')

In [94]:
new_set_id = all_ids.subtract(updated_id_set)
new_set_id.show()

+---+
| id|
+---+
|  3|
+---+



In [96]:
new_set_id = new_set_id.withColumnRenamed('id', 'new_set_id')
new_set_with_id = new_set_id.join(new_set, new_set.id == new_set_id.new_set_id, 'inner').drop('new_set_id')
new_set_with_id.show()

+---------------+---+
|brand_id_string| id|
+---------------+---+
|   [97, 98, 99]|  3|
+---------------+---+



In [99]:
new_set_with_id.show()
updated_sets.show()

+---------------+---+
|brand_id_string| id|
+---------------+---+
|   [97, 98, 99]|  3|
+---------------+---+

+---+----------------+
| id|      brand_sets|
+---+----------------+
|  6|[32, 33, 34, 31]|
|  5| [4, 6, 1, 2, 5]|
|  1|[12, 15, 16, 17]|
|  2|   [10, 7, 8, 9]|
|  4|    [22, 23, 21]|
+---+----------------+



SET IDENTIFICATION USING GRAPH

In [39]:
def open_file(path):
    df = spark.read.format('csv').option('header','true').load(path)
    df = df.withColumn("brand_id_set", split(col('brand_id_set'), ",").cast(ArrayType(IntegerType())).alias("brand_id_set"))
    print(df.count())
    df.show()
    return df

In [34]:
old_cc_path = '/home/preacher/Bungee/CodeRepo/kaushik_dev/bnt/old_brand_id_set.csv'
new_cc_path = '/home/preacher/Bungee/CodeRepo/kaushik_dev/bnt/new_brand_id_set.csv'

In [48]:
old_brand_set = open_file(old_cc_path)
old_brand_set = old_brand_set.withColumn("id", old_brand_set.id.cast(LongType()))
new_brand_set = open_file(new_cc_path)

8
+----+---------------+
|  id|   brand_id_set|
+----+---------------+
|1000|      [6, 2, 1]|
|1001|     [7, 8, 10]|
|1002|   [12, 15, 17]|
|1003|        [9, 11]|
|1004|[100, 101, 102]|
|1005|         [4, 5]|
|1006|   [21, 22, 23]|
|1007|   [31, 32, 33]|
+----+---------------+

7
+----------------+
|    brand_id_set|
+----------------+
| [100, 101, 102]|
|    [97, 98, 99]|
|   [7, 8, 9, 10]|
|[12, 15, 16, 17]|
|          [4, 6]|
|        [22, 23]|
|    [32, 33, 34]|
+----------------+



In [49]:
max_id = get_max_id(old_brand_set)
max_id

1007

In [50]:
new_id = max_id + 1

In [51]:
new_brand_set = insert_id(new_brand_set, new_id)
new_brand_set.show()
new_brand_set.printSchema()

Getting ids for new uuids
+----------------+----+
|    brand_id_set|  id|
+----------------+----+
| [100, 101, 102]|1008|
|    [97, 98, 99]|1009|
|   [7, 8, 9, 10]|1010|
|[12, 15, 16, 17]|1011|
|          [4, 6]|1012|
|        [22, 23]|1013|
|    [32, 33, 34]|1014|
+----------------+----+

root
 |-- brand_id_set: array (nullable = true)
 |    |-- element: integer (containsNull = true)
 |-- id: long (nullable = false)



In [52]:
old_brand_set.printSchema()

root
 |-- id: long (nullable = true)
 |-- brand_id_set: array (nullable = true)
 |    |-- element: integer (containsNull = true)



In [56]:
def get_exploded_set(df, prefix):
    df = df.select(df.id.alias(prefix+'_id'), explode(df.brand_id_set).alias(prefix+'_brand_id'))
    df.show()
    return df

In [ ]:
old_id_brand_id = get_exploded_set(old_brand_set, 'old')

In [ ]:
new_id_brand_id = get_exploded_set(new_brand_set, 'new')

In [59]:
old_to_new_set = new_id_brand_id.join(old_id_brand_id, new_id_brand_id.new_brand_id == old_id_brand_id.old_brand_id, 'inner')

In [ ]:
old_to_new_set.show()

In [ ]:
directed_edges = old_to_new_set.select(old_to_new_set.new_id, old_to_new_set.old_id).dropDuplicates(["new_id","old_id"])
directed_edges.show()

In [82]:
def get_vertex(df):
    id_a = df.select(col("new_id").alias("id"))
    id_b = df.select(col("old_id").alias("id"))
    vertex = combine_dfs([id_a, id_b]).dropDuplicates(['id'])
    return vertex


In [ ]:
vertex = get_vertex(directed_edges)
vertex.show()

In [80]:
def generate_graph_edges(brand_matches):
    directed_edge_a_b = brand_matches.select('new_id','old_id')
    directed_edge_b_a = brand_matches.select(col('new_id').alias('old_id') ,col('old_id').alias('new_id'))
    undirected_edges = combine_dfs([directed_edge_a_b, directed_edge_b_a])
    undirected_edges = undirected_edges.dropDuplicates(["new_id","old_id"]).withColumnRenamed('new_id', 'src').withColumnRenamed('old_id', 'dst')
    return undirected_edges

In [ ]:
edges = generate_graph_edges(directed_edges)
edges.show()

In [83]:
from graphframes import *

In [84]:
graph =  GraphFrame(vertex, edges)

graphframes.graphframe.GraphFrame

In [85]:
sc.setCheckpointDir("/home/preacher/Bungee/bnt/checkpoints/")
cc = graph.connectedComponents()

In [ ]:
cc.show()

In [ ]:
old_id_cc = cc.join(old_id_brand_id, cc.id == old_id_brand_id.old_id, 'inner').drop('id')
old_id_cc = old_id_cc.withColumnRenamed('old_id','id').withColumnRenamed('old_brand_id','brand_id')
old_id_cc.show()
new_id_cc = cc.join(new_id_brand_id, cc.id == new_id_brand_id.new_id, 'inner').drop('id')
new_id_cc = new_id_cc.withColumnRenamed('new_id','id').withColumnRenamed('new_brand_id','brand_id')
new_id_cc.show()

In [97]:
id_cc = combine_dfs([old_id_cc, new_id_cc])

Combning dfs


In [121]:
id_cc.show()

+---------+----+--------+
|component|  id|brand_id|
+---------+----+--------+
|     1002|1002|      17|
|     1002|1002|      15|
|     1002|1002|      12|
|     1000|1000|       1|
|     1000|1000|       2|
|     1000|1000|       6|
|     1007|1007|      33|
|     1007|1007|      32|
|     1007|1007|      31|
|     1000|1005|       5|
|     1000|1005|       4|
|     1001|1001|      10|
|     1001|1001|       8|
|     1001|1001|       7|
|     1004|1004|     102|
|     1004|1004|     101|
|     1004|1004|     100|
|     1006|1006|      23|
|     1006|1006|      22|
|     1006|1006|      21|
+---------+----+--------+
only showing top 20 rows



In [103]:
merge_set_id = id_cc.groupBy('component').agg(collect_set('id').alias('id_sets'), collect_set('brand_id').alias('brand_id_set')).drop('component')
merge_set_id.show()

+------------------+-----------------+
|           id_sets|     brand_id_set|
+------------------+-----------------+
|      [1002, 1011]| [15, 12, 16, 17]|
|[1000, 1005, 1012]|  [1, 5, 2, 6, 4]|
|      [1014, 1007]| [33, 34, 31, 32]|
|[1003, 1001, 1010]|[9, 10, 7, 11, 8]|
|      [1004, 1008]|  [102, 100, 101]|
|      [1006, 1013]|     [21, 22, 23]|
+------------------+-----------------+



In [107]:
updated_brand_set = merge_set_id.withColumn('id',array_min(merge_set_id.id_sets)).drop('id_sets')
updated_brand_set.show()

+-----------------+----+
|     brand_id_set|  id|
+-----------------+----+
| [15, 12, 16, 17]|1002|
|  [1, 5, 2, 6, 4]|1000|
| [33, 34, 31, 32]|1007|
|[9, 10, 7, 11, 8]|1001|
|  [102, 100, 101]|1004|
|     [21, 22, 23]|1006|
+-----------------+----+



In [119]:
old_brand_set = old_brand_set.withColumnRenamed('id', 'old_id').withColumnRenamed('brand_id_set', 'old_brand_id_set')
old_brand_set.show()

+------+----------------+
|old_id|old_brand_id_set|
+------+----------------+
|  1000|       [6, 2, 1]|
|  1001|      [7, 8, 10]|
|  1002|    [12, 15, 17]|
|  1003|         [9, 11]|
|  1004| [100, 101, 102]|
|  1005|          [4, 5]|
|  1006|    [21, 22, 23]|
|  1007|    [31, 32, 33]|
+------+----------------+



In [117]:
updated_and_old_brands = updated_brand_set.join(old_brand_set, old_brand_set.old_id == updated_brand_set.id, 'inner' ).drop('old_id')
updated_and_old_brands.show()

+-----------------+----+----------------+
|     brand_id_set|  id|old_brand_id_set|
+-----------------+----+----------------+
| [15, 12, 16, 17]|1002|    [12, 15, 17]|
|  [1, 5, 2, 6, 4]|1000|       [6, 2, 1]|
| [33, 34, 31, 32]|1007|    [31, 32, 33]|
|[9, 10, 7, 11, 8]|1001|      [7, 8, 10]|
|  [102, 100, 101]|1004| [100, 101, 102]|
|     [21, 22, 23]|1006|    [21, 22, 23]|
+-----------------+----+----------------+



In [125]:
updated_set_ids = updated_and_old_brands.select("id",array_except("brand_id_set","old_brand_id_set").alias('updated_brands')).filter(size(col("updated_brands"))>0)

In [ ]:
old_and_updated_ids = id_cc.select(col('id').alias('old_update_ids')).dropDuplicates()
old_and_updated_ids.show()

In [126]:
new_set_ids = new_brand_set.join(old_and_updated_ids, new_brand_set.id == old_and_updated_ids.old_update_ids, 'leftanti')

In [127]:
updated_set_ids.show()
new_set_ids.show()

+----+--------------+
|  id|updated_brands|
+----+--------------+
|1002|          [16]|
|1000|        [5, 4]|
|1007|          [34]|
|1001|       [9, 11]|
+----+--------------+

+------------+----+
|brand_id_set|  id|
+------------+----+
|[97, 98, 99]|1009|
+------------+----+



22/12/12 22:10:39 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 555798 ms exceeds timeout 120000 ms
22/12/12 22:10:39 WARN SparkContext: Killing executors is not supported by current scheduler.


In [150]:
exact_matches ='/home/preacher/Bungee/bnt/data_set/input.csv'
input = '/home/preacher/Bungee/bnt/data_set/exact_matches.csv'

In [151]:

match_data_warehouse = spark.read.format("csv").option("header","true").load(input)
sim_matches_a_b = spark.read.format("csv").option("header","true").load(exact_matches)

In [175]:
match_data_warehouse.count()

3346933

In [ ]:
match_data_warehouse.show()
sim_matches_a_b.show()

In [153]:
uuid_a = sim_matches_a_b.select(col('uuid_a').alias('uuid')).dropDuplicates()
uuid_b = sim_matches_a_b.select(col('uuid_b').alias('uuid')).dropDuplicates()
uuid = combine_dfs([uuid_a, uuid_b]).dropDuplicates()

Combning dfs


In [154]:
uuid.count()

669473

In [ ]:
uuid.show()

In [164]:
datalake = match_data_warehouse.dropDuplicates(['uuid'])

In [165]:
uuid = uuid.withColumnRenamed('uuid','matches_uuid')

In [166]:
uuid.count()

669473

In [167]:
similar_uuid = datalake.join(uuid, uuid.matches_uuid == datalake.uuid, 'inner')

In [168]:
similar_uuid.printSchema

<bound method DataFrame.printSchema of DataFrame[rid: string, uuid: string, upc: string, source_store: string, product_title: string, store_name: string, source_name: string, brand: string, pack_size: string, product_description: string, sku: string, product_url: string, manufacturer_part_number: string, pack_size_alt: string, custom_sku: string, postal_code: string, quantity: string, source_name_display: string, category: string, subcategory: string, sub_subcategory: string, dimension: string, shipping_weight: string, uom_alt: string, size_alt: string, uom: string, size: string, image_url: string, effective_price: string, parent_sku: string, store_name_display: string, product_segment: string, capture_date: string, sku_uuid: string, domain: string, year: string, month: string, day: string, matches_uuid: string]>

In [171]:
similar_uuid= similar_uuid.drop('matches_uuid')

In [ ]:
similar_uuid.show()

In [176]:
similar_uuid.write.mode("overwrite").option("header",True).csv('/home/preacher/Bungee/bnt/data_set/bungeedatalake/input_data')

In [174]:
similar_uuid.count()

379774

In [177]:
similar_uuid_only = similar_uuid.select('uuid')

In [178]:
sim_matches_a = sim_matches_a_b.join(similar_uuid_only, similar_uuid_only.uuid==sim_matches_a_b.uuid_a, 'inner').drop('uuid')

In [179]:
sim_matches_a.count()

1260211

In [180]:
sim_matches_a_b = sim_matches_a.join(similar_uuid_only, similar_uuid_only.uuid==sim_matches_a.uuid_b, 'inner').drop('uuid')

In [181]:
sim_matches_a_b.count()

912456

In [182]:
sim_matches_a_b.show()

+--------------------+--------------------+--------------------+--------------------+----------+-----------------+-----------------+---------------+------------+-------------+---------------+----+-----+---+
|              uuid_a|              uuid_b|          sku_uuid_a|          sku_uuid_b|     score|base_source_store|comp_source_store|match_timestamp|match_source|         type|        segment|year|month|day|
+--------------------+--------------------+--------------------+--------------------+----------+-----------------+-----------------+---------------+------------+-------------+---------------+----+-----+---+
|001034383213<>386...|00010343832138<>2...|3861385<>walmart<...|24359469<>staples...|       1.0| walmart<>walmart| staples<>staples|     1665964800|         upc|  exact_match|office_supplies|2022|   11| 01|
|11974320827<>1670...|00011974320827<>b...|1670108<>staples<...|b00vbzwuyc<>amazo...|     100.0| staples<>staples|   amazon<>amazon|     1655955182|      manual|similar_mat

In [183]:
sim_matches_a_b.write.mode("overwrite").option("header",True).csv('/home/preacher/Bungee/bnt/data_set/match_data_warehouse/exact_matches')

In [116]:
sim_matches_a_b

DataFrame[uuid_a: string, uuid_b: string, sku_uuid_a: string, sku_uuid_b: string, score: string, base_source_store: string, comp_source_store: string, match_timestamp: string, match_source: string, type: string, segment: string, year: string, month: string, day: string]

In [10]:
bungeedatalake = spark.read.format('csv').option('header','true').load('/home/preacher/Bungee/bnt/data_set/saas_team/brand_info.csv')
bungeedatalake.printSchema()
print(bungeedatalake.count())

root
 |-- uuid: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- source_store: string (nullable = true)

1228847


In [11]:
exact_matches = spark.read.format('csv').option('header','true').load('/home/preacher/Bungee/bnt/data_set/saas_team/exact_matches.csv')
exact_matches.printSchema()
print(exact_matches.count())

root
 |-- uuid_a: string (nullable = true)
 |-- uuid_b: string (nullable = true)
 |-- sku_uuid_a: string (nullable = true)
 |-- sku_uuid_b: string (nullable = true)
 |-- score: string (nullable = true)
 |-- base_source_store: string (nullable = true)
 |-- comp_source_store: string (nullable = true)
 |-- match_timestamp: string (nullable = true)
 |-- match_source: string (nullable = true)
 |-- type: string (nullable = true)
 |-- segment: string (nullable = true)
 |-- year: string (nullable = true)
 |-- month: string (nullable = true)
 |-- day: string (nullable = true)

246866


In [34]:
uuids = bungeedatalake.select('uuid')

In [43]:
uuids.count()

1228847

In [51]:
bungeedatalake = bungeedatalake.dropDuplicates(['uuid'])
bungeedatalake.count()

116927

In [46]:
exact_matches.count()

246866

In [48]:
uuid_a_match = exact_matches.join(uuids, exact_matches.uuid_a == uuids.uuid, 'inner').drop('uuid')
uuid_a_match.printSchema()
print(uuid_a_match.count())

root
 |-- uuid_a: string (nullable = true)
 |-- uuid_b: string (nullable = true)
 |-- sku_uuid_a: string (nullable = true)
 |-- sku_uuid_b: string (nullable = true)
 |-- score: string (nullable = true)
 |-- base_source_store: string (nullable = true)
 |-- comp_source_store: string (nullable = true)
 |-- match_timestamp: string (nullable = true)
 |-- match_source: string (nullable = true)
 |-- type: string (nullable = true)
 |-- segment: string (nullable = true)
 |-- year: string (nullable = true)
 |-- month: string (nullable = true)
 |-- day: string (nullable = true)

146270


In [37]:
uuids.printSchema()

root
 |-- uuid: string (nullable = true)



In [49]:
uuid_a_b_match = uuid_a_match.join(uuids, uuid_a_match.uuid_b == uuids.uuid, 'inner').drop('uuid')
uuid_a_b_match.printSchema()
print(uuid_a_b_match.count())

root
 |-- uuid_a: string (nullable = true)
 |-- uuid_b: string (nullable = true)
 |-- sku_uuid_a: string (nullable = true)
 |-- sku_uuid_b: string (nullable = true)
 |-- score: string (nullable = true)
 |-- base_source_store: string (nullable = true)
 |-- comp_source_store: string (nullable = true)
 |-- match_timestamp: string (nullable = true)
 |-- match_source: string (nullable = true)
 |-- type: string (nullable = true)
 |-- segment: string (nullable = true)
 |-- year: string (nullable = true)
 |-- month: string (nullable = true)
 |-- day: string (nullable = true)



83683


In [50]:
uuid_a_b_match.show()

+--------------------+--------------------+--------------------+--------------------+-----+--------------------+-----------------+---------------+------------+-----------+-------+----+-----+---+
|              uuid_a|              uuid_b|          sku_uuid_a|          sku_uuid_b|score|   base_source_store|comp_source_store|match_timestamp|match_source|       type|segment|year|month|day|
+--------------------+--------------------+--------------------+--------------------+-----+--------------------+-----------------+---------------+------------+-----------+-------+----+-----+---+
|818271010969<>344...|000000152956<>152...|3449348<>petco<>p...|152956<>chewy<>chewy|100.0|        petco<>petco|     chewy<>chewy|     1636987327|      manual|exact_match|   pets|2022|   12| 13|
|800443366110<>300...|000001192247<>119...|3005632<>petco<>p...|119224<>chewy<>chewy|100.0|        petco<>petco|     chewy<>chewy|     1610344020|      manual|not_a_match|   pets|2022|   12| 13|
|sku_603375880<>60...|000

In [17]:
def combine_dfs(df_list):
    print("Combning dfs")
    combined_df = reduce(lambda df1, df2: df1.union(df2.select(df1.columns)), df_list)
    print("Combined df size = {}".format(combined_df.count()))
    return combined_df

In [22]:
bungeedatalake.join(uuid_a, bungeedatalake.uuid == uuid_a.uuid, 'inner').count()

1544672

In [23]:
bungeedatalake.join(uuid_b, bungeedatalake.uuid == uuid_b.uuid, 'inner').count()

1546759

In [25]:
exact_matches.join(bungeedatalake).filter((exact_matches.uuid_a == bungeedatalake.uuid) & (exact_matches.uuid_b == bungeedatalake.uuid)).show()


SyntaxError: invalid syntax (565249398.py, line 1)

In [53]:
bungeedatalake.show()

+--------------------+--------------+------------+
|                uuid|         brand|source_store|
+--------------------+--------------+------------+
|000000152941<>152...|        GoTags| chewy_chewy|
|000000152956<>152...|        GoTags| chewy_chewy|
|000001192247<>119...|        Frisco| chewy_chewy|
|000001192704<>119...|        Frisco| chewy_chewy|
|000002759067<>111...|        OraVet| chewy_chewy|
|000005273343<>146...|    Revolution| chewy_chewy|
|000005274389<>146...|       Epizyme| chewy_chewy|
|000005274413<>146...|   Methimazole| chewy_chewy|
|000005274415<>146...|   Doxycycline| chewy_chewy|
|000005274422<>146...|      Previcox| chewy_chewy|
|000005274440<>146...|       Metacam| chewy_chewy|
|000005274464<>146...|      Clavamox| chewy_chewy|
|000005274516<>146...|   Hydroxyzine| chewy_chewy|
|000005274557<>146...|     Diltiazem| chewy_chewy|
|000005274574<>146...|  Acepromazine| chewy_chewy|
|000005274576<>146...|       Digoxin| chewy_chewy|
|000005274672<>146...|     Simp

In [54]:
bungeedatalake.write.mode("overwrite").option("header",True).csv('/home/preacher/Bungee/bnt/data_set/saas_team/brand_info')

In [57]:
uuid_a_b_match.write.mode("overwrite").option("header",True).csv('/home/preacher/Bungee/bnt/data_set/saas_team/exact_matches')